## Micro GPS adpaté Argos

In [2]:
#pip install ipympl

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import clear_output
from io import BytesIO
from PIL import Image
from IPython.display import HTML

# %matplotlib widget for Voilà compatibility
%matplotlib widget

# Global widgets
file_uploader = widgets.FileUpload(accept='.jpg,.jpeg,.png', multiple=False)
txt_uploader = widgets.FileUpload(accept='.txt', multiple=False)
load_image_button = widgets.Button(description="Load Image", button_style="primary")
report_output = widgets.Output()
status_output = widgets.Output()
plot_output = widgets.Output()
image_report_output = widgets.Output()
export_btn = widgets.Button(description="Save data to txt")
import_btn = widgets.FileUpload(accept='.txt', multiple=False)

# Global variables
fig, ax = None, None
img = None
clicked_points = []
affine_matrices = []
red_annotations = []
green_annotations = []
calibration_df = pd.DataFrame(columns=["label", "Raman_x", "Raman_y"])
measurement_df = pd.DataFrame(columns=["label", "Raman_x", "Raman_y"])

# Additional widgets
click_mode_toggle = widgets.ToggleButton(value=False, description="Add Point (click)")
man_x_input = widgets.FloatText(description='X', step=0.1)
man_y_input = widgets.FloatText(description='Y', step=0.1)
alt_sys_dropdown = widgets.Dropdown(options=["None"], description='System')
man_add_btn = widgets.Button(description="Add Point")
save_btn = widgets.Button(description="Save Image + Report", button_style='info')

coord_input_fields = [widgets.HBox([widgets.FloatText(description=f'X{i+1}'),
                                    widgets.FloatText(description=f'Y{i+1}')]) for i in range(3)]
sys_input = widgets.Text(description='System', placeholder='e.g., µ-Raman')
affine_compute_btn = widgets.Button(description="Compute Transform")
clear_red_btn = widgets.Button(description="Clear Calib", button_style='danger')
clear_green_btn = widgets.Button(description="Clear Meas", button_style='success')

manual_input_layout = widgets.VBox([
    widgets.Label("Add a point using manual coordinates (image or transformed):"),
    widgets.HBox([man_add_btn, man_x_input, man_y_input, alt_sys_dropdown])
])
click_input_layout = widgets.VBox([
    widgets.Label("Add a point by clicking on the image:"),
    click_mode_toggle
])
affine_input_layout = widgets.VBox([
    widgets.Label("Enter measurement coordinates for the 3 selected points:"),
    widgets.HBox([
        affine_compute_btn,
        widgets.VBox(coord_input_fields),
        sys_input
    ])
])
clear_buttons_layout = widgets.HBox([clear_red_btn, clear_green_btn, save_btn])
io_buttons_layout = widgets.HBox([export_btn, import_btn])

widgets_layout = widgets.VBox([
    widgets.HBox([file_uploader, txt_uploader, load_image_button]),
    plot_output,
    manual_input_layout, click_input_layout, affine_input_layout,
    clear_buttons_layout,
    io_buttons_layout,
    report_output,
    status_output,
    image_report_output
])

display(widgets_layout)

def export_points_to_txt(filename="points_export.txt"):
    df = get_full_transformed_dataframe()
    if df.empty:
        display_status("No data to export.")
        return

    # Extract sample_name
    if isinstance(file_uploader.value, dict):
        file_name = list(file_uploader.value.keys())[0]
    elif isinstance(file_uploader.value, (tuple, list)) and len(file_uploader.value) > 0:
        file_name = file_uploader.value[0]['name']
    else:
        file_name = "Unknown"
    sample_name = file_name.split('-')[0]

    try:
        with open(filename, "w", encoding='utf-8') as f:
            f.write(f"# Sample: {sample_name}\n")
            # Write affine matrices info
            for system, matrix in affine_matrices:
                # Flatten matrix row-major
                params = matrix.flatten()
                f.write(f"# AffineSystem: {system} {' '.join(map(str, params))}\n")
            f.write("# label\tRaman_x\tRaman_y")
            for system, _ in affine_matrices:
                f.write(f"\t{system}_x\t{system}_y")
            f.write("\n")
            df.to_csv(f, sep='\t', index=False, float_format="%.4f", header=False)
        display_status(f"✅ Exported point data to {filename}")
    except Exception as e:
        display_status(f"❌ Export failed: {e}")


def import_points_from_txt(change):
    global clicked_points, calibration_df, measurement_df, affine_matrices

    if isinstance(import_btn.value, dict):
        uploaded = list(import_btn.value.values())[0]
    elif isinstance(import_btn.value, (tuple, list)) and len(import_btn.value) > 0:
        uploaded = import_btn.value[0]
    else:
        display_status("❌ Could not read uploaded file.")
        return

    try:
        content = uploaded.get('content')
        if isinstance(content, memoryview):
            content = content.tobytes()
        txt = content.decode('utf-8')
        lines = txt.splitlines()

        affine_matrices.clear()

        # Find the line index where the header starts (line beginning with "# label")
        header_line_index = None
        for i, line in enumerate(lines):
            if line.startswith("# label"):
                header_line_index = i
                break

        if header_line_index is None:
            display_status("❌ Could not find header line '# label...' in file.")
            return

        # Parse affine matrices before the header line
        for line in lines[:header_line_index]:
            if line.startswith("# AffineSystem:"):
                parts = line[len("# AffineSystem:"):].strip().split()
                system = parts[0]
                params = list(map(float, parts[1:7]))
                matrix = np.array(params).reshape(2, 3)
                affine_matrices.append((system, matrix))

        # Extract data lines starting from the header line (remove leading '# ' from header)
        data_lines = [lines[header_line_index][2:]] + lines[header_line_index+1:]

        # Load DataFrame from these lines
        from io import StringIO
        df = pd.read_csv(StringIO("\n".join(data_lines)), sep='\t', comment=None, header=0)

        if not {"label", "Raman_x", "Raman_y"}.issubset(df.columns):
            display_status("❌ Invalid format: must include 'label', 'Raman_x', 'Raman_y'.")
            return

        # Clear old data and annotations
        calibration_df = pd.DataFrame(columns=["label", "Raman_x", "Raman_y"])
        measurement_df = pd.DataFrame(columns=["label", "Raman_x", "Raman_y"])
        clicked_points.clear()
        clear_annotations(red_annotations)
        clear_annotations(green_annotations)

        # Add points and annotations
        for _, row in df.iterrows():
            x, y = row["Raman_x"], row["Raman_y"]
            label = row["label"]
            if label.startswith("CP"):
                clicked_points.append([x, y])
                calibration_df.loc[len(calibration_df)] = [label, x, y]
                annotate_point(x, y, label, 'red')
            else:
                measurement_df.loc[len(measurement_df)] = row[measurement_df.columns.intersection(df.columns)].to_dict()
                annotate_point(x, y, label, 'green')

        # Update dropdowns with loaded affine systems
        alt_sys_dropdown.options = ["None"] + [s for s, _ in affine_matrices]

        update_report()
        display_status(f"✅ Imported {len(df)} points and {len(affine_matrices)} affine systems.")
    except Exception as e:
        display_status(f"❌ Import failed: {e}")

def display_status(msg):
    with status_output:
        clear_output()
        print(msg)

def get_full_transformed_dataframe():
    dfs = []
    for df in [calibration_df, measurement_df]:
        if df.empty:
            continue
        expanded = df.copy()
        for system, matrix in affine_matrices:
            transformed = (matrix @ np.vstack([df["Raman_x"], df["Raman_y"], np.ones(len(df))])).T
            expanded[f"{system}_x"] = np.round(transformed[:, 0], 2)
            expanded[f"{system}_y"] = np.round(transformed[:, 1], 2)
        dfs.append(expanded)
    if dfs:
        return pd.concat(dfs, ignore_index=True)
    return pd.DataFrame()  # empty fallback

def update_report():
    with report_output:
        clear_output()
        full_df = get_full_transformed_dataframe()
        if not full_df.empty:
            display(full_df)

def annotate_point(x, y, label, color='red'):
    point_artist = plt.plot(x, y, '+', color=color)[0]
    label_text = plt.text(
        x, y - 300, label,
        color='white', fontsize=12,
        bbox=dict(boxstyle='round,pad=0.2', facecolor='black', edgecolor='none', alpha=0.3)
    )
    if color == 'red':
        red_annotations.append((point_artist, label_text))
    else:
        green_annotations.append((point_artist, label_text))
    fig.canvas.draw()

def compute_affine_matrix(src_pts, dst_pts):
    A, B = [], []
    for (x, y), (u, v) in zip(src_pts, dst_pts):
        A.extend([[x, y, 1, 0, 0, 0], [0, 0, 0, x, y, 1]])
        B.extend([u, v])
    A = np.array(A)
    B = np.array(B)
    params = np.linalg.lstsq(A, B, rcond=None)[0]
    return np.array([[params[0], params[1], params[2]], [params[3], params[4], params[5]]])

def on_canvas_click(event):
    if not event.inaxes or not click_mode_toggle.value:
        return
    x, y = event.xdata, event.ydata
    if len(clicked_points) < 3:
        label = f"CP{len(clicked_points) + 1}"
        clicked_points.append([x, y])
        annotate_point(x, y, label, 'red')
        calibration_df.loc[len(calibration_df)] = [label, x, y]
    else:
        label = f"MP{len(measurement_df) + 1}"
        transformed = {}
        for system, matrix in affine_matrices:
            t = matrix @ np.array([x, y, 1])
            transformed[f"{system}_x"] = t[0]
            transformed[f"{system}_y"] = t[1]
        annotate_point(x, y, label, 'green')
        measurement_df.loc[len(measurement_df)] = {
            "label": label, "Raman_x": x, "Raman_y": y, **transformed
        }
    update_report()

def on_manual_add(b):
    if alt_sys_dropdown.value != "None":
        system = alt_sys_dropdown.value
        matrix = next((m for s, m in affine_matrices if s == system), None)
        if matrix is None:
            return
        p = np.array([man_x_input.value, man_y_input.value, 1])
        full_matrix = np.vstack([matrix, [0, 0, 1]])
        x, y, _ = np.linalg.inv(full_matrix) @ p
    else:
        x, y = man_x_input.value, man_y_input.value

    if len(clicked_points) < 3:
        label = f"CP{len(clicked_points) + 1}"
        clicked_points.append([x, y])
        annotate_point(x, y, label, 'red')
        calibration_df.loc[len(calibration_df)] = [label, x, y]
    else:
        label = f"MP{len(measurement_df) + 1}"
        transformed = {}
        for system, matrix in affine_matrices:
            t = matrix @ np.array([x, y, 1])
            transformed[f"{system}_x"] = t[0]
            transformed[f"{system}_y"] = t[1]
        annotate_point(x, y, label, 'green')
        measurement_df.loc[len(measurement_df)] = {
            "label": label, "Raman_x": x, "Raman_y": y, **transformed
        }
    update_report()

def on_compute_transform(b):
    if len(clicked_points) != 3:
        return
    real_world_coords = [[box.children[0].value, box.children[1].value] for box in coord_input_fields]
    matrix = compute_affine_matrix(clicked_points, real_world_coords)
    system = sys_input.value.strip()
    if not system:
        return
    affine_matrices.append((system, matrix))
    alt_sys_dropdown.options = ["None"] + [s for s, _ in affine_matrices]

def clear_annotations(annotations):
    for point_artist, label_text in annotations:
        point_artist.remove()
        label_text.remove()
    annotations.clear()
    fig.canvas.draw()

def on_clear_red(b):
    global clicked_points, calibration_df
    clicked_points.clear()
    calibration_df = pd.DataFrame(columns=["label", "Raman_x", "Raman_y"])
    clear_annotations(red_annotations)
    update_report()

def on_clear_green(b):
    global measurement_df
    measurement_df = pd.DataFrame(columns=["label", "Raman_x", "Raman_y"])
    clear_annotations(green_annotations)
    update_report()

def save_image_and_report(filename="voila_output.png"):
    if isinstance(file_uploader.value, dict):
        file_name = list(file_uploader.value.keys())[0]
    elif isinstance(file_uploader.value, (tuple, list)) and len(file_uploader.value) > 0:
        file_name = file_uploader.value[0]['name']
    else:
        file_name = "Unknown"
    sample_name = file_name.split('-')[0]
    
    if fig is None or (calibration_df.empty and measurement_df.empty):
        return

    save_fig = plt.figure(figsize=(8.27, 2.92))  # Half A5 lanscape height
    ax_img = save_fig.add_axes([0.05, 0.2, 0.4, 0.7])
    ax_table = save_fig.add_axes([0.5, 0.05, 0.45, 0.9])

    # --- Load image ---
    file_content = next(iter(file_uploader.value))['content']
    image_pil = Image.open(BytesIO(file_content))
    img = np.array(image_pil)
    img_width, img_height = image_pil.size

    # --- Load TXT and parse coordinates ---
    txt_raw = next(iter(txt_uploader.value))['content']
    if isinstance(txt_raw, memoryview):
        txt_raw = txt_raw.tobytes()
    txt_content = txt_raw.decode('latin-1')
    lines = [line.strip() for line in txt_content.splitlines() if line.strip() and not line.startswith("#")]
    X_coor = [float(x) for x in lines[0].split()]
    Y_coor = [float(line.split()[0]) for line in lines[1:]]
    x_min_real, x_max_real = min(X_coor), max(X_coor)
    y_min_real, y_max_real = min(Y_coor), max(Y_coor)

    # --- Image with correct extent ---
    extent = [x_min_real, x_max_real, y_max_real, y_min_real]
    ax_img.imshow(img, extent=extent, origin='upper')
    ax_img.set_ylim(y_max_real, y_min_real)  # Flip Y-axis
    ax_img.set_xlabel("X (µm)", fontsize=6)
    ax_img.set_ylabel("Y (µm)", fontsize=6)
    ax_img.tick_params(labelsize=4)
    ax_img.set_title(f"Measurement points: {sample_name}", fontsize=6)

    # --- Annotations ---
    for point_artist, label_text in red_annotations + green_annotations:
        ax_img.plot(point_artist.get_xdata(), point_artist.get_ydata(), '+', color=point_artist.get_color(), markersize=4)
        ax_img.text(label_text.get_position()[0], label_text.get_position()[1] - 300, label_text.get_text(),
                    color="white", fontsize=6,
                    bbox=dict(boxstyle='round,pad=0.2', facecolor='black', edgecolor='none', alpha=0.3))

    # --- Create textual table representation ---
    ax_table.axis('off')
    data = get_full_transformed_dataframe()
    numeric_cols = data.select_dtypes(include=[np.number]).columns
    data[numeric_cols] = data[numeric_cols].round(2)

    table_text = data.to_string(index=False)
    ax_table.text(0, 0.95, table_text, ha='left', va='top', fontsize=6, family='monospace')

    # --- Save figure ---
    save_fig.savefig(filename, dpi=300)
    plt.close(save_fig)

    # --- Display saved image ---
    try:
        img_data = Image.open(filename)
        img_array = np.array(img_data)

        with image_report_output:
            image_report_output.clear_output(wait=True)
            fig2 = plt.figure(figsize=(8, 8))
            ax2 = fig2.gca()
            ax2.imshow(img_array)
            ax2.axis('off')
            plt.tight_layout()
            plt.show()

        display_status("Report image displayed below. Right-click to save, or use the download link if available.")
    except Exception as e:
        display_status(f"Error loading saved image: {e}")

def on_load_image_clicked(b):
    global fig, ax, img
    if not file_uploader.value or not txt_uploader.value:
        display_status("Please upload both the image and the coordinate TXT file.")
        return
    
    if isinstance(file_uploader.value, dict):
        file_name = list(file_uploader.value.keys())[0]
    elif isinstance(file_uploader.value, (tuple, list)) and len(file_uploader.value) > 0:
        file_name = file_uploader.value[0]['name']
    else:
        file_name = "Unknown"
    sample_name = file_name.split('-')[0]


    # --- Load image ---
    file_content = next(iter(file_uploader.value))['content']
    image_pil = Image.open(BytesIO(file_content))
    img = np.array(image_pil)
    img_width, img_height = image_pil.size

    # --- Load TXT and parse coordinates ---
    txt_raw = next(iter(txt_uploader.value))['content']
    if isinstance(txt_raw, memoryview):
        txt_raw = txt_raw.tobytes()
    txt_content = txt_raw.decode('latin-1')

    lines = [line.strip() for line in txt_content.splitlines() if line.strip() and not line.startswith("#")]

    X_coor = [float(x) for x in lines[0].split()]
    Y_coor = [float(line.split()[0]) for line in lines[1:]]
    x_min_real, x_max_real = min(X_coor), max(X_coor)
    y_min_real, y_max_real = min(Y_coor), max(Y_coor)

    scale_x = img_width / (x_max_real - x_min_real)
    scale_y = img_height / (y_max_real - y_min_real)

    display_status(
        f"Image loaded.\n"
        f"Real X range: {x_min_real:.2f} µm to {x_max_real:.2f} µm\n"
        f"Real Y range: {y_min_real:.2f} µm to {y_max_real:.2f} µm\n"
        f"Scale X: {scale_x:.3f} px/µm, Scale Y: {scale_y:.3f} px/µm"
    )

    # --- Show image with correct scale ---
    with plot_output:
        clear_output(wait=True)
        fig = plt.figure(figsize=(8, 8))
        ax = fig.gca()
        extent = [x_min_real, x_max_real, y_max_real, y_min_real]  # flip Y for image display
        ax.imshow(img, extent=extent)
        plt.title(f"Measurement points: {sample_name}", fontsize=14, weight='bold')
        plt.xlabel("X (µm)", fontsize=12)
        plt.ylabel("Y (µm)", fontsize=12)
        plt.show()
        fig.canvas.mpl_connect("button_press_event", on_canvas_click)

# Connect buttons
load_image_button.on_click(on_load_image_clicked)
man_add_btn.on_click(on_manual_add)
affine_compute_btn.on_click(on_compute_transform)
clear_red_btn.on_click(on_clear_red)
clear_green_btn.on_click(on_clear_green)
save_btn.on_click(lambda b: save_image_and_report("voila_output.png"))
export_btn.on_click(lambda b: export_points_to_txt("points_export.txt"))
import_btn.observe(import_points_from_txt, names='value')

